[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ourownstory/neural_prophet/blob/master/tutorials/UnderstandeTheBenchmarkingPipeline.ipynb)

# custom evaluation pipeline with min-max normalisation


In [1]:
import pandas as pd
from neuralprophet import set_log_level, set_random_seed
from tot.df_utils import _check_min_df_len, prep_or_copy_df, check_dataframe, handle_missing_data, split_df, return_df_in_original_format, maybe_drop_added_dates
from tot.exp_utils import evaluate_forecast
from tot.models.models_neuralprophet import NeuralProphetModel
from darts.dataprocessing.transformers import Scaler
from darts import TimeSeries
from sklearn.preprocessing import MinMaxScaler

In [2]:
set_log_level("ERROR")

In [3]:
data_location = "https://raw.githubusercontent.com/ourownstory/neuralprophet-data/main/datasets/"
df_air = pd.read_csv(data_location + 'air_passengers.csv')

In [4]:
df_air["y"].shape

(144,)

In [5]:
df_air.head()

,ds,y
0,1949-01-01,112
1,1949-02-01,118
2,1949-03-01,132
3,1949-04-01,129
4,1949-05-01,121


### 1. Data-specific pre-processing

### min-max (0,1) normalization

In [6]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(pd.Series.to_numpy(df_air["y"]).reshape(-1, 1))
df_air["y"] = pd.Series(scaled.flatten())
df_air.head()

,ds,y
0,1949-01-01,0.015444
1,1949-02-01,0.027027
2,1949-03-01,0.054054
3,1949-04-01,0.048263
4,1949-05-01,0.032819


In [7]:
# prep_or_copy_df() ensures that the df has an "ID" column to be usable in the further process
df_air, received_ID_col, received_single_time_series, _ = prep_or_copy_df(df_air)
# check_dataframe() performs a basic sanity check on the data
df_air = check_dataframe(df_air, check_y=True)
# handle_missing_data() imputes missing data
df_air = handle_missing_data(df_air, freq='MS')
# split_df() splits the data into train and test data



df_air_train, df_air_test = split_df(
    df=df_air,
    test_percentage=0.40,
)

### 2. Model definition

In [8]:
# set_random_seed(42)
model_class = NeuralProphetModel
params =  {
    "n_forecasts": 3,
    "n_lags":5,
    "seasonality_mode": "multiplicative",
    "learning_rate": 0.1,
    "_data_params":{},
}
model=model_class(params=params)

### 3. Model-specific data pre-processing

In [9]:
# check if train and test df contain enough samples
_check_min_df_len(df=df_air_train, min_len=model.n_forecasts + model.n_lags)
_check_min_df_len(df=df_air_test, min_len=model.n_forecasts)
# extend the test df with historic values from the train df
df_air_test = model.maybe_extend_df(df_air_train, df_air_test)

### 4. Fit model

In [10]:
model.model.fit(df=df_air_train, freq='MS', progress="none", minimal=True)

### 5. Predict model

In [11]:
# the model-individual predict function outputs the forecasts as a df
fcst_train = model.model.predict(df=df_air_train)
fcst_test = model.model.predict(df=df_air_test)

### 6. Model-specific post-processing:

In [12]:
# As you can see, the method is a class method and hence linked to the model
fcst_test = model.maybe_drop_added_values_from_df(fcst_test, df_air_test)

### 7. Data-specific data post-processing:

In [13]:
fcst_test.head()

,ds,y,ID,yhat1,yhat2,yhat3,ar1,ar2,ar3,trend,season_yearly
0,1956-04-01,0.403475,__df__,0.414815,NaN,NaN,-0.122413,NaN,NaN,0.555009,-0.017782
1,1956-05-01,0.413127,__df__,0.410462,0.411167,NaN,-0.119227,-0.118522,NaN,0.564943,-0.035254
2,1956-06-01,0.521236,__df__,0.529903,0.527977,0.527969,-0.125646,-0.127572,-0.127580,0.575207,0.080342
3,1956-07-01,0.596525,__df__,0.632518,0.630251,0.630301,-0.143303,-0.145570,-0.145520,0.585140,0.190681
4,1956-08-01,0.581081,__df__,0.616770,0.614271,0.613672,-0.137226,-0.139725,-0.140324,0.595405,0.158591


In [14]:
fcst_test_inverse_y = scaler.inverse_transform(pd.Series.to_numpy(fcst_test["y"]).reshape(-1, 1))
fcst_test["y"] = pd.Series(fcst_test_inverse_y.flatten())
fcst_test.head()

,ds,y,ID,yhat1,yhat2,yhat3,ar1,ar2,ar3,trend,season_yearly
0,1956-04-01,313.0,__df__,0.414815,NaN,NaN,-0.122413,NaN,NaN,0.555009,-0.017782
1,1956-05-01,318.0,__df__,0.410462,0.411167,NaN,-0.119227,-0.118522,NaN,0.564943,-0.035254
2,1956-06-01,374.0,__df__,0.529903,0.527977,0.527969,-0.125646,-0.127572,-0.127580,0.575207,0.080342
3,1956-07-01,413.0,__df__,0.632518,0.630251,0.630301,-0.143303,-0.145570,-0.145520,0.585140,0.190681
4,1956-08-01,405.0,__df__,0.616770,0.614271,0.613672,-0.137226,-0.139725,-0.140324,0.595405,0.158591


In [15]:
fcst_test_inverse_y = scaler.inverse_transform(pd.Series.to_numpy(fcst_test["yhat1"]).reshape(-1, 1))
fcst_test["yhat1"] = pd.Series(fcst_test_inverse_y.flatten())
fcst_test.head()

,ds,y,ID,yhat1,yhat2,yhat3,ar1,ar2,ar3,trend,season_yearly
0,1956-04-01,313.0,__df__,318.874020,NaN,NaN,-0.122413,NaN,NaN,0.555009,-0.017782
1,1956-05-01,318.0,__df__,316.619312,0.411167,NaN,-0.119227,-0.118522,NaN,0.564943,-0.035254
2,1956-06-01,374.0,__df__,378.489566,0.527977,0.527969,-0.125646,-0.127572,-0.127580,0.575207,0.080342
3,1956-07-01,413.0,__df__,431.644444,0.630251,0.630301,-0.143303,-0.145570,-0.145520,0.585140,0.190681
4,1956-08-01,405.0,__df__,423.486968,0.614271,0.613672,-0.137226,-0.139725,-0.140324,0.595405,0.158591


In [16]:
fcst_test_inverse_y = scaler.inverse_transform(pd.Series.to_numpy(fcst_test["yhat2"]).reshape(-1, 1))
fcst_test["yhat2"] = pd.Series(fcst_test_inverse_y.flatten())
fcst_test.head()

,ds,y,ID,yhat1,yhat2,yhat3,ar1,ar2,ar3,trend,season_yearly
0,1956-04-01,313.0,__df__,318.874020,NaN,NaN,-0.122413,NaN,NaN,0.555009,-0.017782
1,1956-05-01,318.0,__df__,316.619312,316.984674,NaN,-0.119227,-0.118522,NaN,0.564943,-0.035254
2,1956-06-01,374.0,__df__,378.489566,377.491865,0.527969,-0.125646,-0.127572,-0.127580,0.575207,0.080342
3,1956-07-01,413.0,__df__,431.644444,430.469920,0.630301,-0.143303,-0.145570,-0.145520,0.585140,0.190681
4,1956-08-01,405.0,__df__,423.486968,422.192525,0.613672,-0.137226,-0.139725,-0.140324,0.595405,0.158591


In [17]:
fcst_test_inverse_y = scaler.inverse_transform(pd.Series.to_numpy(fcst_test["yhat3"]).reshape(-1, 1))
fcst_test["yhat3"] = pd.Series(fcst_test_inverse_y.flatten())
fcst_test.head()

,ds,y,ID,yhat1,yhat2,yhat3,ar1,ar2,ar3,trend,season_yearly
0,1956-04-01,313.0,__df__,318.874020,NaN,NaN,-0.122413,NaN,NaN,0.555009,-0.017782
1,1956-05-01,318.0,__df__,316.619312,316.984674,NaN,-0.119227,-0.118522,NaN,0.564943,-0.035254
2,1956-06-01,374.0,__df__,378.489566,377.491865,377.488067,-0.125646,-0.127572,-0.127580,0.575207,0.080342
3,1956-07-01,413.0,__df__,431.644444,430.469920,430.496133,-0.143303,-0.145570,-0.145520,0.585140,0.190681
4,1956-08-01,405.0,__df__,423.486968,422.192525,421.881951,-0.137226,-0.139725,-0.140324,0.595405,0.158591


In [18]:
# in case, missing data was imputed maybe_drop_added_dates() removes it again
# fcst_train_df, df_air_train = maybe_drop_added_dates(fcst_train_df, df_air_train)
# fcst_test_df, df_air_test = maybe_drop_added_dates(fcst_test_df, df_air_test)

### 8. Evaluation:

In [19]:
# evaluate_forecast() computes the selected error metrics
result_train, result_test = evaluate_forecast(fcst_train, fcst_test, metrics=['MAPE','MAE','RMSE'], metadata=None)
print(result_test)

WARNING - (py.warnings._showwarnmsg) - /Users/pediatria/Desktop/TUMai/test-of-time/tot/evaluation/metrics.py:81: RuntimeWarning: divide by zero encountered in divide
  error_relative = np.abs(np.divide(error, truth))

WARNING - (py.warnings._showwarnmsg) - /Users/pediatria/Desktop/TUMai/test-of-time/tot/evaluation/metrics.py:81: RuntimeWarning: divide by zero encountered in divide
  error_relative = np.abs(np.divide(error, truth))



{'MAPE': 9.854049, 'MAE': 42.019135, 'RMSE': 51.641846}
